In [ ]:
"""DISTILBERT.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1zgdmSQx89GWepVH-kRgK_4a8mtTPqNJ-
"""

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
!pip install -U transformers

In [ ]:
import os

In [ ]:
!pip install -U transformers datasets
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Define all file names and labels
files = {
    "twitter_parsed_dataset.csv": "twitter",
    "twitter_racism_parsed_dataset.csv": "racism",
    "twitter_sexism_parsed_dataset.csv": "sexism",
    "youtube_parsed_dataset.csv": "youtube"
}

In [ ]:
all_dfs = []

In [ ]:
for file, label in files.items():
    df = pd.read_csv(file)
    text_col = None
    for col in df.columns:
        if 'text' in col.lower():
            text_col = col
            break
    if not text_col:
        raise ValueError(f"Couldn't find a text column in {file}")
    df = df[[text_col]].dropna().copy()
    df.columns = ['text']
    df['label'] = label
    all_dfs.append(df)

In [ ]:
final_df = pd.concat(all_dfs).reset_index(drop=True)
final_df.head()

In [ ]:
# Sample 2000 entries per label to balance the dataset
sample_df = final_df.groupby('label').apply(lambda x: x.sample(n=2000, random_state=42)).reset_index(drop=True)

In [ ]:
# Encode labels into integers
le = LabelEncoder()
sample_df['label_encoded'] = le.fit_transform(sample_df['label'])

In [ ]:
# Check results
sample_df[['label', 'label_encoded']].drop_duplicates()

In [ ]:
# Split into train and validation sets (90/10 split)
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sample_df['text'].tolist(),
    sample_df['label_encoded'].tolist(),
    test_size=0.1,
    stratify=sample_df['label_encoded'],
    random_state=42
)

In [ ]:
# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# Tokenize train and validation sets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import Dataset

In [ ]:
# Rebuild train and validation into HF-compatible Dataset format
train_dict = {
    'text': train_texts,
    'label': train_labels
}

In [ ]:
val_dict = {
    'text': val_texts,
    'label': val_labels
}

In [ ]:
train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

In [ ]:
def tokenize_fn(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)

In [ ]:
# Rename label column for Trainer compatibility
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

In [ ]:
# Set format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import DistilBertForSequenceClassification

In [ ]:
# Number of unique labels (classes)
num_labels = len(set(train_labels))

In [ ]:
# Load pretrained DistilBERT model with classification head
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50
)

In [ ]:
from transformers import DistilBertForSequenceClassification
num_labels = len(set(train_labels))  # or len(set(sample_df['label_encoded']))
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)

In [ ]:
from transformers import Trainer, EvalPrediction
import numpy as np
from sklearn.metrics import accuracy_score
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

In [ ]:
# Evaluate model on validation set
trainer.evaluate()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(final_df['label'])

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
%reset -f

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
!pip install -U transformers

In [ ]:
import os

In [ ]:
!pip install -U transformers datasets
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Define all file names and labels
files = {
    "toxicity_parsed_dataset.csv": "toxicity",
    "twitter_racism_parsed_dataset.csv": "racism",
    "twitter_sexism_parsed_dataset.csv": "sexism",
    "youtube_parsed_dataset.csv": "youtube"
}

In [ ]:
all_dfs = []

In [ ]:
for file, label in files.items():
    df = pd.read_csv(file)
    text_col = None
    for col in df.columns:
        if 'text' in col.lower():
            text_col = col
            break
    if not text_col:
        raise ValueError(f"Couldn't find a text column in {file}")
    df = df[[text_col]].dropna().copy()
    df.columns = ['text']
    df['label'] = label
    all_dfs.append(df)

In [ ]:
final_df = pd.concat(all_dfs).reset_index(drop=True)
final_df.head()

In [ ]:
from sklearn.utils import resample
import pandas as pd

In [ ]:
# Get the smallest class count
min_count = 1970

In [ ]:
# Balanced list
balanced = []

In [ ]:
# Loop through each label and resample
for label in final_df["label"].unique():
    label_df = final_df[final_df["label"] == label]
    if len(label_df) > min_count:
        resampled = resample(label_df, replace=False, n_samples=min_count, random_state=42)
    else:
        resampled = resample(label_df, replace=True, n_samples=min_count, random_state=42)
    balanced.append(resampled)

In [ ]:
# Concatenate all balanced samples
balanced_df = pd.concat(balanced).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Check new distribution
print("New Label Distribution:")
print(balanced_df["label"].value_counts())

In [ ]:
# New label encoding
label_map = {'toxicity': 0, 'racism': 1, 'youtube': 2, 'sexism': 3}
balanced_df['label_encoded'] = balanced_df['label'].map(label_map)

In [ ]:
import re

In [ ]:
def clean(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

In [ ]:
balanced_df["clean_text"] = balanced_df["text"].apply(clean)

In [ ]:
# Split into train and validation sets (90/10 split)
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    balanced_df['text'].tolist(),
    balanced_df['label_encoded'].tolist(),
    test_size=0.1,
    stratify=balanced_df['label_encoded'],
    random_state=42
)

In [ ]:
# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# Tokenize train and validation sets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import Dataset

In [ ]:
# Rebuild train and validation into HF-compatible Dataset format
train_dict = {
    'text': train_texts,
    'label': train_labels
}

In [ ]:
val_dict = {
    'text': val_texts,
    'label': val_labels
}

In [ ]:
train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

In [ ]:
def tokenize_fn(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)

In [ ]:
# Rename label column for Trainer compatibility
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

In [ ]:
# Set format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import DistilBertForSequenceClassification# Number of unique labels (classes)num_labels = len(set(train_labels))# Load pretrained DistilBERT model with classification headmodel = DistilBertForSequenceClassification.from_pretrained(    'distilbert-base-uncased',    num_labels=num_labels)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="no"
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "macro_f1": f1_score(labels, preds, average='macro')
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
# Evaluate model on validation set
trainer.evaluate()

In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
val_encodings = tokenizer(
    val_texts,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

In [ ]:
import torch

In [ ]:
val_labels_tensor = torch.tensor(val_labels)

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class ValDataset(Dataset):
    def _init_(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def _len_(self):
        return len(self.labels)

    def _getitem_(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'label': self.labels[idx]
        }

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
all_preds = []
all_labels = []

In [ ]:
model.to(device)
model.eval()

In [ ]:
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)


        label_key = 'label' if 'label' in batch else 'labels'
        labels = batch[label_key].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
# Define class names (edit if needed)
label_names = ['toxicity', 'racism', 'youtube', 'sexism']

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names)

In [ ]:
# Plot
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=label_names))

In [ ]:
model.save_pretrained("distilbert_model/")
tokenizer.save_pretrained("distilbert_model/")